## Request 1: Join RACM_A and RACM_B to compare if there are changes in start-date, end-date or both and split into multiple csv files

## Request 2: Add GIM database to the joint RACM data from request 1
  
## Request 3: Join RACM_A to GIM; Join RACM_B to GIM
* Have the option to export one csv with all information or split into multiple csvs - one for each country

# Plan:
### 1. Create functions that can be called to perform a specific task 
Functions: 
    a. To create "Key" by joining UPN and ConFig columns. Reason: each row of data has unique UPN and ConFig #s, if the record is repeated twice, it means that there is a change in etiher the start or end date. 
    b. To set index to Key for dataframe
    c. Join the RACM_A dataframe to the RACM_B dataframe
    d. Compare the Start Date with End Date 
    e. Separate data for each country
    f. Export data into excel. Each country will have its own excel (92) + (1) master record
### 2. Request 1:Join RACM_A and RACM_B to compare if start date and/or end date changed and split into multiple csv files
    a. Import datasets 
    b. Create a column Key to join with the second dataset.
    c. Create a column IPN by comning UPN with ConFig. This will be a temporary product registration code.
    d. Retain RACM string data for process checking
    e. Remove the empty column
    f. Change the date format
    g. Remove the leading "0" from UPN so that it can be set as index for joining GIM data. GIM["ItemId'] does not have the leading "0". 
    h. Join the two datasets. 
    i. Export the data as one file
    j. Export the data as multiple files (one per country)
    k. Create a dataframe for only rows that have different start-date and/or end_date
### 3. Request 2: Add GIM database to the joint RACM data from request 1
    a. Import GIM data
    b. Change te Catalog Number column to string
    c. Set CatalogNumber as index but also keep the CatalogNumber column in the data
    d. Select columns of interest
    e. Left join GIM with the combined RACM so RACM does not get dropped in case GIM data is missing
    f. Export the data as one file
### 4. Request 3: Join RACM_A to GIM and Join RACM_B to GIM
Both RACM datasets are joined to GIM individually by the following steps:
    a. RACM: Set UPN as index
    b. GIM: Set CatalogNumber as index
    c. Join the RACM with GIM by index
    d. Export data as individual files

# 2. List of Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, math
from collections import Counter

# 3. List of Functions

In [2]:
def date_change(ddmmyyyy):
    # This function changes dd-mm-yyyy to mm-dd-yyyy format
    x = ddmmyyyy.split("/")
    return x[1]+"/"+x[0]+"/"+x[2]

In [3]:
def read_csv(file_path):
    df = pd.read_csv(file_path, sep='|', names=['Country Code', 'UPN', 'ConFig', 'Empty', 'Start Date', 'End Date'])
    # Create ConFig column
    df["Key"] = df["Country Code"] + df["UPN"] + df["ConFig"]
    df["IPN"] = df["UPN"] + '-' + df["ConFig"]
    # Add the original data
    df_ = pd.read_csv(file_path, names = ["Original Data"])
    df["Original Data"] = df_["Original Data"]
    del df["Empty"]
    # Change the date format
    df["Start Date"] = df["Start Date"].apply(date_change)
    df["End Date"] = df["End Date"].apply(date_change)
    # Remove "0" in front of any UPN that starts with "0" before combining with GIM columns because GIM[Catalog]\
    # does not have the leading "0". 
    df["UPN"] = [s.lstrip("0") for s in df["UPN"]]
    # Set key as index and drop the key
    df = df.set_index("Key", drop = True)
    return df

In [4]:
def choose_country_code(row):
    if row["Country Code_A"]=="":
        return row["Country Code_B"]
    else:
        return row["Country Code_A"]

def choose_UPN(row):
    if row["UPN_A"]=="":
        return row["UPN_B"]
    else:
        return row["UPN_A"]
    
def choose_IPN(row):
    if row["IPN_A"]=="":
        return row["IPN_B"]
    else:
        return row["IPN_A"]

def frequency_graph(s):
    """
    This finction will generate a bar graph. X: Country, Y: Frequency (number of time the data is processed)
    input : s: pandas.Series
    """
    s = RACM_AB["Country Code"]
    x = Counter(list(s))
    y = list(x.items())
    y.sort(key=lambda x: -x[1])
    plt.figure(figsize=(25,5))
    plt.bar([val[0] for val in y], [val[1] for val in y])
    plt.xlabel('Country Code')
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()

# 4. Import Data

## 4.1 Dataset for RACM_A

Create dataframes for each excel, label the columns, and remove empty columns

In [6]:
# Import and create column names ## OLDEST FILE
RACM_A = pd.read_csv(r"C:\Users\kpham\Desktop\12OCT2022 - TAN_CODE\RACMUPDATE_12OCT2022.txt", sep='|', names=['Country Code', 'UPN', 'ConFig', 'Empty', 'Start Date', 'End Date'])
# Create Key and IPN columns
RACM_A["Key"] = RACM_A["Country Code"]+ RACM_A["UPN"]+ "|" + RACM_A["ConFig"]
RACM_A["IPN"] = RACM_A["UPN"] + '-' + RACM_A["ConFig"]
# Add the original data
RACM_A_ = pd.read_csv(r"C:\Users\kpham\Desktop\12OCT2022 - TAN_CODE\RACMUPDATE_12OCT2022.txt", names = ["Original Data"])
RACM_A["Original Data"] = RACM_A_["Original Data"]
del RACM_A["Empty"]
# Change the date format
RACM_A["Start Date"] = RACM_A["Start Date"].apply(date_change)
RACM_A["End Date"] = RACM_A["End Date"].apply(date_change)
# Remove "0" in front of any UPN that starts with "0" before combining with GIM columns because GIM[Catalog] does not have \
# the leading "0". 
RACM_A["UPN"] = [s.lstrip("0") for s in RACM_A["UPN"]]
# Set key as index and drop the key
RACM_A = RACM_A.set_index("Key", drop = True)
RACM_A

,Country Code,UPN,ConFig,Start Date,End Date,IPN,Original Data
Key,,,,,,,
AE077.0082|C1,AE,77.0082,C1,04/05/2021,01/26/2025,077.0082-C1,AE|077.0082|C1||05/04/2021|26/01/2025
AR077.0082|C1,AR,77.0082,C1,12/23/2020,12/31/9999,077.0082-C1,AR|077.0082|C1||23/12/2020|31/12/9999
AR077.0082|C2,AR,77.0082,C2,08/02/2022,12/31/9999,077.0082-C2,AR|077.0082|C2||02/08/2022|31/12/9999
AT077.0082|C1,AT,77.0082,C1,12/23/2020,05/26/2025,077.0082-C1,AT|077.0082|C1||23/12/2020|26/05/2025
AT077.0082|C2,AT,77.0082,C2,10/28/2021,12/31/9999,077.0082-C2,AT|077.0082|C2||28/10/2021|31/12/9999
...,...,...,...,...,...,...,...
USSSUP300STR|S3,US,SSUP300STR,S3,08/29/2022,12/31/9999,SSUP300STR-S3,US|SSUP300STR|S3||29/08/2022|31/12/9999
XISSUP300STR|S2,XI,SSUP300STR,S2,05/20/2021,12/31/9999,SSUP300STR-S2,XI|SSUP300STR|S2||20/05/2021|31/12/9999
XISSUP300STR|S3,XI,SSUP300STR,S3,08/29/2022,12/31/9999,SSUP300STR-S3,XI|SSUP300STR|S3||29/08/2022|31/12/9999


## 4.2 Dataset for RACM_B

In [7]:
# Import and create column names ## NEWEST FILE
RACM_B = pd.read_csv(r"C:\Users\kpham\Desktop\12OCT2022 - TAN_CODE\RACMUPDATE_22NOV2022.txt", sep='|', names=['Country Code', 'UPN', 'ConFig', 'Empty', 'Start Date', 'End Date'])
# Create Key and IPN columns
RACM_B["Key"] = RACM_B["Country Code"] + RACM_B["UPN"] + "|" + RACM_B["ConFig"]
RACM_B["IPN"] = RACM_B["UPN"] + '-' + RACM_B["ConFig"]
# Add the original data
RACM_B_ = pd.read_csv(r"C:\Users\kpham\Desktop\12OCT2022 - TAN_CODE\RACMUPDATE_22NOV2022.txt", names = ["Original Data"])
RACM_B["Original Data"] = RACM_B_["Original Data"]
del RACM_B["Empty"]
# Change the date format
RACM_B["Start Date"] = RACM_B["Start Date"].apply(date_change)
RACM_B["End Date"] = RACM_B["End Date"].apply(date_change)
# Remove "0" infront of any UPN that starts with "0" before combining with GIM columns because GIM[Catalog] does not have the leading "0". 
RACM_B["UPN"] = [s.lstrip("0") for s in RACM_B["UPN"]]
# Set key as index and drop the key
RACM_B = RACM_B.set_index("Key", drop = True)
RACM_B

,Country Code,UPN,ConFig,Start Date,End Date,IPN,Original Data
Key,,,,,,,
AE077.0082|C1,AE,77.0082,C1,04/05/2021,01/26/2025,077.0082-C1,AE|077.0082|C1||05/04/2021|26/01/2025
AR077.0082|C1,AR,77.0082,C1,12/23/2020,12/31/9999,077.0082-C1,AR|077.0082|C1||23/12/2020|31/12/9999
AR077.0082|C2,AR,77.0082,C2,08/02/2022,12/31/9999,077.0082-C2,AR|077.0082|C2||02/08/2022|31/12/9999
AT077.0082|C1,AT,77.0082,C1,12/23/2020,05/26/2025,077.0082-C1,AT|077.0082|C1||23/12/2020|26/05/2025
AT077.0082|C2,AT,77.0082,C2,10/28/2021,12/31/9999,077.0082-C2,AT|077.0082|C2||28/10/2021|31/12/9999
...,...,...,...,...,...,...,...
USSSUP300STR|S3,US,SSUP300STR,S3,08/29/2022,12/31/9999,SSUP300STR-S3,US|SSUP300STR|S3||29/08/2022|31/12/9999
XISSUP300STR|S2,XI,SSUP300STR,S2,05/20/2021,12/31/9999,SSUP300STR-S2,XI|SSUP300STR|S2||20/05/2021|31/12/9999
XISSUP300STR|S3,XI,SSUP300STR,S3,08/29/2022,12/31/9999,SSUP300STR-S3,XI|SSUP300STR|S3||29/08/2022|31/12/9999


## 4.3 Dataset for GIM

In [9]:
# Import GIM data
GIM = pd.read_csv(r"C:\Users\kpham\Desktop\12OCT2022 - TAN_CODE\GIM_11OCT2022.csv", sep=',')
#Change the Catalog Number column to string
GIM["CatalogNumber_index"] = GIM["CatalogNumber"].astype(str)
#Set CatalogNumber as index but also keep the CatalogNumber column in the data
GIM = GIM.set_index(["CatalogNumber_index"], drop = False)
# Select columns of interest
GIM = GIM[["ItemId", "CatalogNumber", "ItemType", "LongDescription"]]
GIM['ItemId']=GIM['ItemId']

GIM
GIM.info()
#GIM['CatalogNumber'].unique()
#for i in GIM['CatalogNumber']:
    #print(i)


<class 'pandas.core.frame.DataFrame'>
Index: 12828 entries, H965100430 to XTG5414510
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ItemId           12828 non-null  int64 
 1   CatalogNumber    12828 non-null  object
 2   ItemType         11773 non-null  object
 3   LongDescription  11929 non-null  object
dtypes: int64(1), object(3)
memory usage: 501.1+ KB


In [ ]:
### Print dataset loaded for GIM
# path = "C:\\Users\\TNguyen5\\OneDrive - Stryker\\PYTHON\\Comparison\\Print_GIM\\GIM_PrintCurrentDataset.csv"
# GIM.to_csv(path, index = True)

# 6. Request 1:Join RACM_A and RACM_B to compare if start date and/or end date changed. Split into multiple csv files.

## 6.1 Combine RACM_A and RACM_B Data

Purpose: to create a linkage between GIM and RACM. GIM: GIM ID, Item Type and Long Description to be added as additional columns to RACM

In [ ]:
# Join two dataframes(RACM_A & RACM_B) with RACM_A on the left and RACM_B on the right
RACM_AB = RACM_A.join(RACM_B, how='outer', lsuffix="_A", rsuffix="_B")

#################################

# Add a new column to capture the Start_Date change in RACM_A & RACM_B
RACM_AB['Start Date Changed'] = (RACM_AB['Start Date_A']!=RACM_AB['Start Date_B'])

# Add a new column to capture the End_Date change in RACM_A & RACM_B
RACM_AB['End Date Changed'] = (RACM_AB['End Date_A']!=RACM_AB['End Date_B'])

# Add a new column to capture if a product was removed from RACM_B, but existed in RACM_A
RACM_AB['Product Removed'] = RACM_AB['UPN_B'].isna()

# Add a new column to capture if a product was added to RACM_B, but was not in RACM_A
RACM_AB['Product Added'] = RACM_AB['UPN_A'].isna()

#Add a new column to capture if the raw data has changed between two files
RACM_AB['RAW Data Changed'] = (RACM_AB['Original Data_A'] != RACM_AB['Original Data_B'])

# Select only columns with either start_date change, end_date change or both. This is for information only.
df_RACM_AB_Comparison = RACM_AB[RACM_AB['Start Date Changed']|RACM_AB['End Date Changed']]

####################################
# Create a column name Index to retain the information when exporting the data. Note: indexes in general will not be exported
RACM_AB['Index'] = RACM_AB.index
RACM_AB["Index"].astype(str)

#Create Country Code column that have data from both RACM_A and RACM_B
RACM_AB["Country Code"] = RACM_AB["Index"].apply(lambda y: y[0:2])

RACM_AB["Index"] = RACM_AB["Index"].apply(lambda y: y[2:])
RACM_AB


In [ ]:
#RACM_AB["Index"] = RACM_AB["Index"].apply(lambda x: x.lstrip("0"))
RACM_AB["Index_GIM"] = RACM_AB["Index"].apply(lambda z: z.split("|", 1)[0])
RACM_AB["Index_GIM"] = [s.lstrip("0") for s in RACM_AB["Index_GIM"]] # Remove the leading "0" since GIM does not ahev "0"
RACM_AB = RACM_AB.set_index("Index_GIM")

In [ ]:
RACM_AB

In [ ]:
# RACM_AB["IPN"] = RACM_AB["Index"].apply(lambda z: z.replace("|", "-"))#.apply(lambda y: y[2:]).apply(lambda z: z.replace("|", "-"))
# RACM_AB

In [ ]:
#######edit with Kieu/Tan 06/02/2021#########

# Create IPN and UPN columns that have data from both RACM_A and RACM_B
RACM_AB["IPN"] = RACM_AB["Index"].apply(lambda z: z.replace("|", "-"))#.apply(lambda y: y[2:]).apply(lambda z: z.replace("|", "-"))

RACM_AB["UPN"] = RACM_AB["Index"].apply(lambda z: z.split("|", 1)[0])#.apply(lambda x: x.lstrip("0"))
RACM_AB["ConFig"] = RACM_AB["Index"].apply(lambda z: z.split("|", 1)[1])
#RACM_AB.drop(columns = ["IPN_A", "UPN_A", "UPN_B", "IPN_B"], inplace = True)

RACM_AB['AB_Index'] = RACM_AB.index

# Get a list of all unique countries for exporting by each country individually for graphing
#RACM_AB['Country Code'] = RACM_AB.apply(choose_country_code, axis=1)
country_list = RACM_AB["Country Code"].unique()
print(len(country_list))
country_list

RACM_AB['Country Code'] = RACM_AB['Country Code'].astype(str)
List_of_Countries = RACM_AB['Country Code'].unique()

# Change data from object to string to retain the leading "0"
RACM_AB['UPN_A'] = RACM_AB['UPN_A'].apply(str)
RACM_AB['UPN_B'] = RACM_AB['UPN_B'].apply(str)
RACM_AB['IPN_A'] = RACM_AB['IPN_A'].apply(str)
RACM_AB['IPN_B'] = RACM_AB['IPN_B'].apply(str)

RACM_AB

# 7. Request 2: Add GIM database to the joint RACM data from request 1

## 7.1 Join two dataframes(RACM_AB & GIM)

In [ ]:
GIM

In [ ]:
RACM_AB

In [ ]:
#Join two dataframes(RACM_AB & GIM) with RACM_AB on the left and GIM on the right

RACM_AB_GIM = RACM_AB.join(GIM)
RACM_AB_GIM["index"] = RACM_AB_GIM.index

RACM_AB_GIM.info()

## 7.2 Export data as a single file

In [ ]:
path = r"C:\Users\TNguyen5\OneDrive - Stryker\PYTHON\Comparison\Print_Dataset_RACM_Combined\RACM_AB.csv"
RACM_AB_GIM.to_csv(path, index = True)

RACM_AB_GIM.info()

# 8. Request 3: Join RACM_A GIM and Join RACM_B to GIM

## 8.1 Reset Index

In [ ]:
#Reset index
RACM_A = pd.read_csv(r"C:\Users\TNguyen5\OneDrive - Stryker\PYTHON\Comparison\Dataset_RACM_A_Oldest\RACMUPDATE.txt", sep='|', names=['Country Code', 'UPN', 'ConFig', 'Empty', 'Start Date', 'End Date'])

#Take away leading "0" from the UPN. GIM data does not have leading "0" in their CatalogNumber(UPN)
RACM_A["UPN"]= RACM_A["UPN"].apply(lambda x: x.lstrip("0"))
RACM_A["Original Data"] = RACM_A_["Original Data"]

#Rest the index. This will help join with GIM data 
RACM_A.set_index("UPN",inplace = True)


RACM_A

## 8.2 Join RACM_A to GIM

In [ ]:
RACM_A_GIM = RACM_A.join(GIM)
RACM_A_GIM


## 8.3 Join RACM_B to GIM

In [ ]:
#Reset index
RACM_B = pd.read_csv(r"C:\Users\TNguyen5\OneDrive - Stryker\PYTHON\Comparison\Dataset_RACM_B_Newest\RACMUPDATE.txt", sep='|', names=['Country Code', 'UPN', 'ConFig', 'Empty', 'Start Date', 'End Date'])

#Take away leading "0" from the UPN. GIM data does not have leading "0" in their CatalogNumber(UPN)
RACM_B["UPN"]= RACM_B["UPN"].apply(lambda x: x.lstrip("0"))

####Added in 05JAN2022 to have the original data show up..... 
RACM_B["Original Data"] = RACM_B_["Original Data"]
###########


#Rest the index. This will help join with GIM data 
RACM_B.set_index("UPN",inplace = True)


RACM_B

In [ ]:
RACM_B_GIM = RACM_B.join(GIM)
RACM_B_GIM

In [ ]:
GIM

## 8.4 Export RACM Data with GIM Data

In [ ]:
# Export RACM A + GIM Dataset
path = "C:\\Users\\TNguyen5\\OneDrive - Stryker\\PYTHON\\Comparison\\Print_Dataset_RACM_A_GIM\\RACM_A_GIM.csv"
RACM_A_GIM.to_csv(path, index = False)

In [ ]:
# Export RACM B + GIM Dataset
path = "C:\\Users\\TNguyen5\\OneDrive - Stryker\\PYTHON\\Comparison\\Print_Dataset_RACM_B_GIM\\RACM_B_GIM.csv"
RACM_B_GIM.to_csv(path, index = False)

In [ ]:
# Export RACM A + GIM Dataset split into Multiple Files
for X in List_of_Countries:
    path = "C:\\Users\\TNguyen5\\OneDrive - Stryker\\PYTHON\\Comparison\\Print_Dataset_RACM_A_GIM_CountrySplit\\" + X + "_GIM.csv"
    X = RACM_A_GIM[(RACM_A_GIM["Country Code"]==X)] 
    X.to_csv(path, index = False)  

In [ ]:
# Export RACM B + GIM Dataset split into Multiple Files
for X in List_of_Countries:
    path = "C:\\Users\\TNguyen5\\OneDrive - Stryker\\PYTHON\\Comparison\\Print_Dataset_RACM_B_GIM_CountrySplit\\" + X + "_GIM.csv"
    X = RACM_B_GIM[(RACM_B_GIM["Country Code"]==X)] 
    X.to_csv(path, index = False)  

In [ ]:
# Export RACM_AB_GIM Dataset Multiple Files
for X in List_of_Countries:
    path = "C:\\Users\\TNguyen5\\OneDrive - Stryker\\PYTHON\\Comparison\\Print_Dataset_RACM_AB_GIM_CountrySplit\\" + X + "_GIM.csv"
    X = RACM_AB_GIM[(RACM_AB_GIM["Country Code"]==X)] 
    X.to_csv(path, index = False)  

In [ ]:
# Export RACM Dataset #2 Multiple Files
for X in List_of_Countries:
    path = "C:\\Users\\kcheung1\\Code\\Project - Quickbase\\Export_all\\" + X + ".csv"
    X = RAC_042021_GIM[(RAC_042021_GIM["Country Code"]==X)] 
    X.to_csv(path, index = False)  

THE END